<h1> Facility Location Problem </h1>

In [ ]:
from IPython.display import IFrame
IFrame("./FacilityLocationProblem_2019.pdf", width=1000, height=600)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Mathematical-Formulation" data-toc-modified-id="Mathematical-Formulation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Mathematical Formulation</a></span></li><li><span><a href="#Install-Required-Packages-and-define-Instance" data-toc-modified-id="Install-Required-Packages-and-define-Instance-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Install Required Packages and define Instance</a></span><ul class="toc-item"><li><span><a href="#Instance-Specifics" data-toc-modified-id="Instance-Specifics-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Instance Specifics</a></span></li></ul></li><li><span><a href="#Set-up-Model" data-toc-modified-id="Set-up-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set up Model</a></span></li><li><span><a href="#Set-up-Decision-Variables" data-toc-modified-id="Set-up-Decision-Variables-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Set up Decision Variables</a></span></li><li><span><a href="#Express-The-Constraints" data-toc-modified-id="Express-The-Constraints-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Express The Constraints</a></span><ul class="toc-item"><li><span><a href="#Not-more-than-N_DC-DC's" data-toc-modified-id="Not-more-than-N_DC-DC's-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Not more than N_DC DC's</a></span></li><li><span><a href="#Satisfy-Demand,-and-Flow-Conservation" data-toc-modified-id="Satisfy-Demand,-and-Flow-Conservation-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Satisfy Demand, and Flow Conservation</a></span></li><li><span><a href="#Satisfy-Production-Capacity" data-toc-modified-id="Satisfy-Production-Capacity-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Satisfy Production Capacity</a></span></li></ul></li><li><span><a href="#Define-The-Objective-Function" data-toc-modified-id="Define-The-Objective-Function-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Define The Objective Function</a></span></li><li><span><a href="#Solve" data-toc-modified-id="Solve-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Solve</a></span></li><li><span><a href="#Further-Analyse-of-the-Solution" data-toc-modified-id="Further-Analyse-of-the-Solution-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Further Analyse of the Solution</a></span></li><li><span><a href="#Render-A-Visualization" data-toc-modified-id="Render-A-Visualization-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Render A Visualization</a></span></li><li><span><a href="#Download-the-code" data-toc-modified-id="Download-the-code-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Download the code</a></span></li></ul></div>

## Mathematical Formulation

min $\sum_{i = 1}^n \sum_{j = 1}^n c_{ij}X_{SHIP, ij} + \sum_{i=1}^n f_i X_{DC, i}$

st.

Not more than $N_{DC}$ locations: $\sum_{i = 1}^n X_{DC, i} <= N_{DC}$ 

Satisfy the demand-constraint: $\sum_{i=1}^m X_{SHIP, ij} = D[j] \qquad \qquad\qquad \forall j$

Non-negative shipping: $X_{SHIP, ij} >= 0\qquad \qquad \qquad \quad \qquad\qquad \forall i,j$

No cheating: $X_{SHIP, ii} = 0 \qquad \qquad \qquad  \qquad \qquad \qquad\qquad \quad \forall i$

Satisfy the Supply-constraint: $ S[i]*X_{DC, i} >= \sum_{j = 1}^m X_{SHIP, i,j} \quad \quad \quad \forall i$

## Install Required Packages and define Instance

In [2]:
import pandas as pd
import numpy as np
import sys

try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')      

from docplex.mp.environment import Environment
from docplex.mp.model import Model
#env = Environment()
#env.print_information()

In [3]:
url = open("URL").read()
key = open("APIKEY").read()

### Instance Specifics

In [4]:
c = pd.read_csv("./FacilityLocationDistances.csv", sep = ";").dropna()
c.index = c['Distance (km)']
c = c.drop(['Distance (km)'], axis = 1)
cities = list(c.columns)
N_DC = 37

D = [100 for i in range(len(cities))]
F = [150000 for i in range(len(cities))]
S = [4000 for i in range(len(cities))]

model_name = "Facility Location Problem"

In [5]:
c

,Amsterdam,Antwerp,Athens,Barcelona,Berlin,Bern,Brussels,Calais,Cologne,Copenhagen,...,Prague,Rome,Rotterdam,Strasbourg,Stuttgart,The Hague,Turin,Venice,Vienna,Zurich
Distance (km),,,,,,,,,,,,,,,,,,,,,
Amsterdam,0.0,160.0,3082.0,1639.0,649.0,875.0,209.0,385.0,280.0,904.0,...,973.0,1835.0,80.0,683.0,703.0,56.0,1264.0,1449.0,1196.0,861.0
Antwerp,160.0,0.0,2766.0,1465.0,723.0,704.0,46.0,211.0,237.0,861.0,...,870.0,1660.0,100.0,544.0,659.0,139.0,1090.0,1275.0,1180.0,687.0
Athens,3082.0,2766.0,0.0,3312.0,2552.0,2627.0,3021.0,2976.0,2562.0,3414.0,...,2198.0,2551.0,2826.0,2581.0,2428.0,3061.0,2250.0,1995.0,1886.0,2449.0
Barcelona,1639.0,1465.0,3312.0,0.0,1899.0,913.0,1419.0,1399.0,1539.0,2230.0,...,1679.0,1471.0,1565.0,1072.0,1263.0,1589.0,892.0,1327.0,1989.0,1036.0
Berlin,649.0,723.0,2552.0,1899.0,0.0,986.0,782.0,936.0,575.0,743.0,...,354.0,1573.0,697.0,801.0,636.0,712.0,1172.0,1108.0,666.0,863.0
Bern,875.0,704.0,2627.0,913.0,986.0,0.0,655.0,854.0,583.0,1392.0,...,766.0,897.0,802.0,232.0,350.0,825.0,312.0,642.0,907.0,123.0
Brussels,209.0,46.0,3021.0,1419.0,782.0,655.0,0.0,212.0,219.0,1035.0,...,911.0,1615.0,146.0,488.0,641.0,170.0,1044.0,1229.0,1134.0,641.0
Calais,385.0,211.0,2976.0,1399.0,936.0,854.0,212.0,0.0,431.0,1075.0,...,1082.0,1738.0,311.0,627.0,792.0,352.0,1110.0,1352.0,1346.0,764.0
Cologne,280.0,237.0,2562.0,1539.0,575.0,583.0,219.0,431.0,0.0,730.0,...,659.0,1586.0,254.0,402.0,395.0,283.0,1015.0,1072.0,915.0,612.0


## Set up Model

In [6]:
mdl = Model(model_name)

In [7]:
from docplex.mp.context import Context
context = Context.make_default_context()

context.solver.docloud.url = url
context.solver.docloud.key = key
context.solver.agent = 'docloud'

## Set up Decision Variables

In [8]:
X_DC = mdl.binary_var_dict(cities, name="is_DC__")
X_ship = mdl.integer_var_matrix(c, cities, name = "IsActive__")
X_prod = mdl.integer_var_dict(cities,  name = "prod___")

In [9]:
X_prod

{'Amsterdam': docplex.mp.Var(type=I,name='prod____Amsterdam'),
 'Antwerp': docplex.mp.Var(type=I,name='prod____Antwerp'),
 'Athens': docplex.mp.Var(type=I,name='prod____Athens'),
 'Barcelona': docplex.mp.Var(type=I,name='prod____Barcelona'),
 'Berlin': docplex.mp.Var(type=I,name='prod____Berlin'),
 'Bern': docplex.mp.Var(type=I,name='prod____Bern'),
 'Brussels': docplex.mp.Var(type=I,name='prod____Brussels'),
 'Calais': docplex.mp.Var(type=I,name='prod____Calais'),
 'Cologne': docplex.mp.Var(type=I,name='prod____Cologne'),
 'Copenhagen': docplex.mp.Var(type=I,name='prod____Copenhagen'),
 'Edinburgh': docplex.mp.Var(type=I,name='prod____Edinburgh'),
 'Frankfurt': docplex.mp.Var(type=I,name='prod____Frankfurt'),
 'Geneva': docplex.mp.Var(type=I,name='prod____Geneva'),
 'Genoa': docplex.mp.Var(type=I,name='prod____Genoa'),
 'Hamburg': docplex.mp.Var(type=I,name='prod____Hamburg'),
 'Le Havre': docplex.mp.Var(type=I,name='prod____Le Havre'),
 'Lisbon': docplex.mp.Var(type=I,name='prod____L

## Express The Constraints

### Not more than N_DC DC's

In [10]:
mdl.add_constraint(mdl.sum(X_DC) <= N_DC)

docplex.mp.LinearConstraint[](is_DC___Amsterdam+is_DC___Antwerp+is_DC___Athens+is_DC___Barcelona+is_DC___Berlin+is_DC___Bern+is_DC___Brussels+is_DC___Calais+is_DC___Cologne+is_DC___Copenhagen+is_DC___Edinburgh+is_DC___Frankfurt+is_DC___Geneva+is_DC___Genoa+is_DC___Hamburg+is_DC___Le Havre+is_DC___Lisbon+is_DC___London+is_DC___Luxembourg+is_DC___Lyon+is_DC___Madrid+is_DC___Marseille+is_DC___Milan+is_DC___Munich+is_DC___Naples+is_DC___Nice+is_DC___Paris+is_DC___Prague+is_DC___Rome+is_DC___Rotterdam+is_DC___Strasbourg+is_DC___Stuttgart+is_DC___The Hague+is_DC___Turin+is_DC___Venice+is_DC___Vienna+is_DC___Zurich,LE,37)

### Satisfy Demand, and Flow Conservation

In [11]:
for i in range(len(cities)):
    for j in range(len(cities)):
        mdl.add_constraint(X_ship[(cities[i],cities[j])] >= 0)
    mdl.add_constraint(D[i]
                       == mdl.sum(X_ship[(cities[j],cities[i])] for j in range(len(cities))) - \
                           mdl.sum(X_ship[(cities[i],cities[j])] for j in range(len(cities))) + \
                           X_prod[(cities[i])])

In [12]:
X_DC

{'Amsterdam': docplex.mp.Var(type=B,name='is_DC___Amsterdam'),
 'Antwerp': docplex.mp.Var(type=B,name='is_DC___Antwerp'),
 'Athens': docplex.mp.Var(type=B,name='is_DC___Athens'),
 'Barcelona': docplex.mp.Var(type=B,name='is_DC___Barcelona'),
 'Berlin': docplex.mp.Var(type=B,name='is_DC___Berlin'),
 'Bern': docplex.mp.Var(type=B,name='is_DC___Bern'),
 'Brussels': docplex.mp.Var(type=B,name='is_DC___Brussels'),
 'Calais': docplex.mp.Var(type=B,name='is_DC___Calais'),
 'Cologne': docplex.mp.Var(type=B,name='is_DC___Cologne'),
 'Copenhagen': docplex.mp.Var(type=B,name='is_DC___Copenhagen'),
 'Edinburgh': docplex.mp.Var(type=B,name='is_DC___Edinburgh'),
 'Frankfurt': docplex.mp.Var(type=B,name='is_DC___Frankfurt'),
 'Geneva': docplex.mp.Var(type=B,name='is_DC___Geneva'),
 'Genoa': docplex.mp.Var(type=B,name='is_DC___Genoa'),
 'Hamburg': docplex.mp.Var(type=B,name='is_DC___Hamburg'),
 'Le Havre': docplex.mp.Var(type=B,name='is_DC___Le Havre'),
 'Lisbon': docplex.mp.Var(type=B,name='is_DC___L

### Satisfy Production Capacity

In [13]:
for i in range(len(cities)):
    #mdl.add_constraint(S[i]*X_DC[(cities[i])] >= mdl.sum(X_ship[(cities[i],cities[k])] for k in range(len(cities))))
    mdl.add_constraint(S[i]*X_DC[(cities[i])] >= X_prod[(cities[i])])

## Define The Objective Function

In [14]:
mdl.minimize( \
    (mdl.sum(X_DC[(cities[i])]*F[i] for i in range(len(cities)))) + \
    (mdl.sum(X_ship[(cities[i], cities[j])]*c.iloc[i,j] for i in range(len(cities)) for j in range(len(cities)))))

In [15]:
mdl.prettyprint()

// This file has been generated by DOcplex
// model name is: Facility Location Problem
// var contrainer section
dvar bool is_DC__[37];
dvar int IsActive__[37][37];
dvar int prod___[37];

minimize
 150000 is_DC___Amsterdam + 150000 is_DC___Antwerp + 150000 is_DC___Athens
 + 150000 is_DC___Barcelona + 150000 is_DC___Berlin + 150000 is_DC___Bern
 + 150000 is_DC___Brussels + 150000 is_DC___Calais + 150000 is_DC___Cologne
 + 150000 is_DC___Copenhagen + 150000 is_DC___Edinburgh
 + 150000 is_DC___Frankfurt + 150000 is_DC___Geneva + 150000 is_DC___Genoa
 + 150000 is_DC___Hamburg + 150000 is_DC___Le_Havre + 150000 is_DC___Lisbon
 + 150000 is_DC___London + 150000 is_DC___Luxembourg + 150000 is_DC___Lyon
 + 150000 is_DC___Madrid + 150000 is_DC___Marseille + 150000 is_DC___Milan
 + 150000 is_DC___Munich + 150000 is_DC___Naples + 150000 is_DC___Nice
 + 150000 is_DC___Paris + 150000 is_DC___Prague + 150000 is_DC___Rome
 + 150000 is_DC___Rotterdam + 150000 is_DC___Strasbourg
 + 150000 is_DC___Stuttg

 IsActive___Copenhagen_The_Hague >= 0;
 IsActive___Copenhagen_Turin >= 0;
 IsActive___Copenhagen_Venice >= 0;
 IsActive___Copenhagen_Vienna >= 0;
 IsActive___Copenhagen_Zurich >= 0;
 IsActive___Amsterdam_Copenhagen + IsActive___Antwerp_Copenhagen
 + IsActive___Athens_Copenhagen + IsActive___Barcelona_Copenhagen
 + IsActive___Berlin_Copenhagen + IsActive___Bern_Copenhagen
 + IsActive___Brussels_Copenhagen + IsActive___Calais_Copenhagen
 + IsActive___Cologne_Copenhagen + IsActive___Edinburgh_Copenhagen
 + IsActive___Frankfurt_Copenhagen + IsActive___Geneva_Copenhagen
 + IsActive___Genoa_Copenhagen + IsActive___Hamburg_Copenhagen
 + IsActive___Le_Havre_Copenhagen + IsActive___Lisbon_Copenhagen
 + IsActive___London_Copenhagen + IsActive___Luxembourg_Copenhagen
 + IsActive___Lyon_Copenhagen + IsActive___Madrid_Copenhagen
 + IsActive___Marseille_Copenhagen + IsActive___Milan_Copenhagen
 + IsActive___Munich_Copenhagen + IsActive___Naples_Copenhagen
 + IsActive___Nice_Copenhagen + IsActive___P

 IsActive___Nice_Bern >= 0;
 IsActive___Nice_Brussels >= 0;
 IsActive___Nice_Calais >= 0;
 IsActive___Nice_Cologne >= 0;
 IsActive___Nice_Copenhagen >= 0;
 IsActive___Nice_Edinburgh >= 0;
 IsActive___Nice_Frankfurt >= 0;
 IsActive___Nice_Geneva >= 0;
 IsActive___Nice_Genoa >= 0;
 IsActive___Nice_Hamburg >= 0;
 IsActive___Nice_Le_Havre >= 0;
 IsActive___Nice_Lisbon >= 0;
 IsActive___Nice_London >= 0;
 IsActive___Nice_Luxembourg >= 0;
 IsActive___Nice_Lyon >= 0;
 IsActive___Nice_Madrid >= 0;
 IsActive___Nice_Marseille >= 0;
 IsActive___Nice_Milan >= 0;
 IsActive___Nice_Munich >= 0;
 IsActive___Nice_Naples >= 0;
 IsActive___Nice_Nice >= 0;
 IsActive___Nice_Paris >= 0;
 IsActive___Nice_Prague >= 0;
 IsActive___Nice_Rome >= 0;
 IsActive___Nice_Rotterdam >= 0;
 IsActive___Nice_Strasbourg >= 0;
 IsActive___Nice_Stuttgart >= 0;
 IsActive___Nice_The_Hague >= 0;
 IsActive___Nice_Turin >= 0;
 IsActive___Nice_Venice >= 0;
 IsActive___Nice_Vienna >= 0;
 IsActive___Nice_Zurich >= 0;
 IsActive___Amst

 - IsActive___Zurich_Copenhagen - IsActive___Zurich_Edinburgh
 - IsActive___Zurich_Frankfurt - IsActive___Zurich_Geneva
 - IsActive___Zurich_Genoa - IsActive___Zurich_Hamburg
 - IsActive___Zurich_Le_Havre - IsActive___Zurich_Lisbon
 - IsActive___Zurich_London - IsActive___Zurich_Luxembourg
 - IsActive___Zurich_Lyon - IsActive___Zurich_Madrid
 - IsActive___Zurich_Marseille - IsActive___Zurich_Milan
 - IsActive___Zurich_Munich - IsActive___Zurich_Naples
 - IsActive___Zurich_Nice - IsActive___Zurich_Paris - IsActive___Zurich_Prague
 - IsActive___Zurich_Rome - IsActive___Zurich_Rotterdam
 - IsActive___Zurich_Strasbourg - IsActive___Zurich_Stuttgart
 - IsActive___Zurich_The_Hague - IsActive___Zurich_Turin
 - IsActive___Zurich_Venice - IsActive___Zurich_Vienna + prod____Zurich == 100
 ;
 4000 is_DC___Amsterdam >= prod____Amsterdam;
 4000 is_DC___Antwerp >= prod____Antwerp;
 4000 is_DC___Athens >= prod____Athens;
 4000 is_DC___Barcelona >= prod____Barcelona;
 4000 is_DC___Berlin >= prod____Be

##  Solve

In [16]:
mdl.print_information()

Model: Facility Location Problem
 - number of variables: 1443
   - binary=37, integer=1406, continuous=0
 - number of constraints: 1444
   - linear=1444
 - parameters: defaults


In [17]:
print(mdl.solve(url=url, key=key))

solution for: Facility Location Problem
objective: 2.0047e+06
is_DC___Amsterdam=0
is_DC___Antwerp=0
is_DC___Athens=1
is_DC___Barcelona=0
is_DC___Berlin=1
is_DC___Bern=0
is_DC___Brussels=1
is_DC___Calais=0
is_DC___Cologne=0
is_DC___Edinburgh=0
is_DC___Frankfurt=0
is_DC___Geneva=0
is_DC___Genoa=1
is_DC___Le Havre=0
is_DC___Lisbon=0
is_DC___Lyon=0
is_DC___Madrid=1
is_DC___Marseille=0
is_DC___Nice=0
is_DC___Prague=0
is_DC___Strasbourg=0
is_DC___Turin=0
is_DC___Vienna=0
is_DC___Zurich=1
IsActive___Amsterdam_Hamburg=0
IsActive___Amsterdam_The Hague=0
IsActive___Antwerp_Amsterdam=100
IsActive___Antwerp_Brussels=0
IsActive___Antwerp_Hamburg=0
IsActive___Antwerp_Rotterdam=100
IsActive___Antwerp_The Hague=0
IsActive___Barcelona_Madrid=0
IsActive___Barcelona_Marseille=0
IsActive___Berlin_Hamburg=200
IsActive___Berlin_Prague=200
IsActive___Bern_Geneva=100
IsActive___Bern_Turin=0
IsActive___Bern_Zurich=0
IsActive___Brussels_Antwerp=300
IsActive___Brussels_Calais=100
IsActive___Brussels_Cologne=200


## Further Analyse of the Solution

In [18]:
print(mdl.report())
print(mdl.solve_details)

* model Facility Location Problem solved with objective = 2004700.000
None
status  = integer optimal, tolerance
time    = 8.10694 s.
problem = MILP
gap     = 0.00999235%



In [19]:
## convert your array into a dataframe
VALS = mdl.solution.get_all_values()
ARCS = pd.DataFrame(np.reshape(VALS[len(cities):(len(VALS)-len(cities))], (len(cities), len(cities))), index = cities, columns = cities)
PROD = pd.DataFrame(np.reshape(VALS[(len(VALS) - len(cities)):len(VALS)], (len(cities), 1)), index = cities, columns = ["Production"])
FACS = pd.DataFrame(np.reshape(VALS[0:len(cities)], (len(cities), 1)), index = cities, columns = ["Facility"])

In [20]:
FACS[FACS['Facility'] == 1]

,Facility
Athens,1.0
Berlin,1.0
Brussels,1.0
Genoa,1.0
Madrid,1.0
Zurich,1.0


In [21]:
ARCS

,Amsterdam,Antwerp,Athens,Barcelona,Berlin,Bern,Brussels,Calais,Cologne,Copenhagen,...,Prague,Rome,Rotterdam,Strasbourg,Stuttgart,The Hague,Turin,Venice,Vienna,Zurich
Amsterdam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Antwerp,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Athens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Barcelona,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Berlin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Brussels,0.0,300.0,0.0,0.0,0.0,0.0,0.0,100.0,200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
Calais,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cologne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Copenhagen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
PROD

,Production
Amsterdam,0.0
Antwerp,0.0
Athens,100.0
Barcelona,0.0
Berlin,500.0
Bern,0.0
Brussels,1300.0
Calais,0.0
Cologne,0.0
Copenhagen,0.0


In [23]:
print(mdl.parameters.mip.prettyprint())

parameters.mip={
    display:int(2!s)
    interval:int(2!s)
    ordertype:int(2!s)
    parameters.mip.cuts={
        bqp:int(2!s)
        cliques:int(2!s)
        covers:int(2!s)
        disjunctive:int(2!s)
        flowcovers:int(2!s)
        gomory:int(2!s)
        gubcovers:int(2!s)
        implied:int(2!s)
        liftproj:int(2!s)
        localimplied:int(2!s)
        mcfcut:int(2!s)
        mircut:int(2!s)
        pathcut:int(2!s)
        rlt:int(2!s)
        zerohalfcut:int(2!s)
    }
    parameters.mip.limits={
        aggforcut:int(2!s)
        auxrootthreads:int(2!s)
        cutpasses:int(2!s)
        cutsfactor:num(2!s)
        eachcutlimit:int(2!s)
        gomorycand:int(2!s)
        gomorypass:int(2!s)
        nodes:int(2!s)
        polishtime:num(2!s)
        populate:int(2!s)
        probedettime:num(2!s)
        probetime:num(2!s)
        repairtries:int(2!s)
        solutions:int(2!s)
        strongcand:int(2!s)
        strongit:int(2!s)
        treememory:num(2!s)
   

## Render A Visualization

In [25]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
locs = pd.DataFrame(columns = ["City", "Latitude", "Longitude"])

for i in range(len(cities)):
    location = geolocator.geocode(cities[i])
    locs.loc[i] = [cities[i], location.latitude,location.longitude]

locs.to_csv("coordinates_out.csv")

FACS_locs = pd.DataFrame(columns = ["City", "Latitude", "Longitude"])
FACS_cities = list(FACS[FACS['Facility'] == 1].index)
for i in range(len(FACS_cities)):
    location = geolocator.geocode(FACS_cities[i])
    FACS_locs.loc[i] = [FACS_cities[i], location.latitude,location.longitude]

FACS_locs.to_csv("FACScoordinates_out.csv")

## Download the code

In [28]:
print("git clone 'https://github.com/TomKennes/Facility_Location_Problem__docplex__python.git'")

git clone 'https://github.com/TomKennes/Facility_Location_Problem__docplex__python.git'
